https://docs.aws.amazon.com/sagemaker/latest/dg/mxnet-example1.html

https://github.com/aws/sagemaker-python-sdk

http://sagemaker.readthedocs.io/en/latest/

In [ ]:
import mxnet as mx
import numpy as np
from sagemaker.mxnet import MXNet
from sagemaker import get_execution_role
from io import BytesIO
import boto3

First, we define the Job that 

In [ ]:
role = get_execution_role()

mnist_estimator = MXNet(entry_point='part2_sm_mnist.py',
                        py_version='py3',
                        role=role,
                        train_instance_count=1, 
                        train_instance_type='ml.p2.xlarge')

Now we can call the fit() method. This does a few things behind the scenes, but of note:
1. This fit() method's input argument is an S3 address (or address in a dictionary). The method then automatically downloads the file(s) onto the instance to a preset directory. Recall from part2 that these file locations are in the 'channel_input_dirs' argument going into the training script's train() method.
2. This method begins a Sagemaker Job, which you can find on the [Sagemaker Console](https://console.aws.amazon.com/sagemaker/home), under the "Jobs" tab on the left.
3. Once the job is complete, a Model will be created, which you can also find on the Sagemaker Console.

In [ ]:
#mnist_estimator.fit("s3://jakechenawspublic/sample_data/mnist/train/")
mnist_estimator.fit(inputs={'images':'s3://jakechenawspublic/sample_data/mnist/train/images',
                            'labels':'s3://jakechenawspublic/sample_data/mnist/train/labels'})

Now we can deploy this model.

In [ ]:
predictor = mnist_estimator.deploy(instance_type="ml.m4.xlarge",
                                   initial_instance_count=1,
                                   endpoint_name='tutorial-mnist-endpoint')

Done! Let's now run a record through the predictor.

In [ ]:
X_test = np.loadtxt('images_sm.csv', delimiter=',').reshape(-1, 1, 28, 28)

y_proba = np.array(predictor.predict(X_test))

y_proba.shape

Looks good so far. 

In [ ]:
mx.nd.argmax(mx.nd.array(y_proba), axis=1)

Once our predictor is deployed, any future application can take advantage of the prediction endpoint we created above. This allows us to close this development notebook and stop paying for the resources.

While we won't do that right now, we can demonstrate this by wiping the 'predictor' object from memory.

In [ ]:
del(predictor)
predictor.predict(X_test)

Now, let's feed the records through the predictor we deployed from scratch. This can be done by invoking the endpoint using Boto3

In [ ]:
s3 = boto3.client('s3')
sm = boto3.client('sagemaker')
sm_runtime = boto3.client('sagemaker-runtime')

First, let's load the new records from S3 into memory.

In [ ]:
# Load the image records from S3 into local memory
resp = s3.get_object(
    Bucket='jakechenawspublic',
    Key='sample_data/mnist/test/images/images_sm.csv'
)

X_test = resp['Body'].read()   

Next, let's get the name of the prediction endpoint. If you forgot what this is, you can look it up on the [Sagemaker Console](https://console.aws.amazon.com/sagemaker/home) or by using Boto3 to list out what they are.

In [ ]:
# Get prediction endpoint
resp = sm.list_endpoints()
for r in resp['Endpoints']:
    print(r['EndpointName'])

As shown above, our endpoint name is 'tutorial-mnist-predictor'. Let's now invoke this endpoint.

Sagemaker's endpoint API, by default, is set up to receive json or csv. For more info on how input data is processed by default, or on how to customize your own, see [this page](https://github.com/aws/sagemaker-python-sdk#input-processing).

We will be using Sagemaker's default CSV input. To do this, we modify both the "ContentType" and "Accept" parameters to 'text/csv', as shown below. For more information on these parameters, see the [Boto3 documentation for invoke_endpoint()](http://boto3.readthedocs.io/en/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint)

In [ ]:
# Send the dataset to the prediction endpoint
resp = sm_runtime.invoke_endpoint(
    EndpointName='tutorial-mnist-predictor',
    Body=X_test,
    ContentType='text/csv', # -- These 2 are not required by the docs but very helpful.
    Accept='text/csv'       # /  Not sure why the values would be different.
)

In [ ]:
# Load the response body back into an Numpy array for further processing
b = BytesIO(resp['Body'].read())
array = np.loadtxt(b, delimiter=',')
array.shape

The shape looks good. Now let's see what the predicted numbers are:

In [ ]:
mx.nd.argmax(mx.nd.array(array), axis=1)

Congratulations on reaching the end of this tutorial. To finish up, let's clean up the resources used here so we stop paying for the prediction endpoint. You can do that in the [Sagemaker Console](https://console.aws.amazon.com/sagemaker/home), or using the Boto3 API as demonstrated below.

In [ ]:
resp = sm.delete_endpoint(EndpointName='tutorial-mnist-predictor')
print(resp['ResponseMetadata']['HTTPStatusCode'])